## ⚠️ 메모리 절약 팁

8GB GPU에서 크래시가 발생하면:
1. 다른 GPU 사용 프로그램 종료
2. 아래 명령어 실행: `nvidia-smi` 로 GPU 메모리 확인
3. 여전히 크래시 시 → **더 가벼운 BLIP-1 사용 추천**

In [2]:
# 필요한 라이브러리 import
import torch
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import os
import json
from pathlib import Path
import time

print("✅ 라이브러리 import 완료")

✅ 라이브러리 import 완료


## 1. GPU 환경 확인

In [3]:
print("=" * 70)
print("🔍 GPU 환경 진단")
print("=" * 70)

print(f"\n✓ CUDA 사용 가능: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✓ GPU 이름: {torch.cuda.get_device_name(0)}")
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✓ GPU 메모리: {total_memory:.2f} GB")
    print(f"✓ PyTorch 버전: {torch.__version__}")
    print(f"✓ CUDA 버전: {torch.version.cuda}")
else:
    print("\n⚠️  CUDA를 사용할 수 없습니다! CPU 모드로 실행됩니다 (매우 느림)")

print("\n" + "=" * 70)

🔍 GPU 환경 진단

✓ CUDA 사용 가능: True
✓ GPU 이름: NVIDIA GeForce RTX 4060
✓ GPU 메모리: 8.00 GB
✓ PyTorch 버전: 2.9.1+cu128
✓ CUDA 버전: 12.8



## 2. BLIP-2 모델 로드

BLIP-2-OPT-2.7B 모델을 사용합니다 (Qwen2-VL보다 훨씬 가벼움)

In [4]:
model_name = "Salesforce/blip2-opt-2.7b"

print("🔄 BLIP-2 모델 로드 중...")
print("⚠️  첫 실행 시 모델 다운로드로 시간이 소요될 수 있습니다 (~5GB)")

# 프로세서 로드
processor = Blip2Processor.from_pretrained(model_name)

# 모델 로드 (8bit 양자화로 메모리 절약)
model = Blip2ForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_8bit=True  # 메모리 절약
)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ 모델 로드 완료 (디바이스: {device})")

if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1024**3
    print(f"✅ GPU 메모리 사용: {allocated:.2f} GB")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


🔄 BLIP-2 모델 로드 중...
⚠️  첫 실행 시 모델 다운로드로 시간이 소요될 수 있습니다 (~5GB)


`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Exception in thread Thread-6 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\subprocess.py", line 1515, in _readerthread
    buffe

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 모델 로드 완료 (디바이스: cuda)
✅ GPU 메모리 사용: 3.92 GB


## 3. 이미지 캡셔닝 함수 정의

In [5]:
def generate_detailed_caption(image_path, max_size=512):
    """
    이미지에 대한 상세한 캡션을 생성합니다.
    
    Args:
        image_path: 이미지 파일 경로
        max_size: 이미지 최대 크기
        
    Returns:
        dict: category와 caption을 포함한 결과
    """
    try:
        start_time = time.time()
        
        # 이미지 로드
        image = Image.open(image_path).convert('RGB')
        
        # 이미지 리사이징
        width, height = image.size
        if max(width, height) > max_size:
            ratio = max_size / max(width, height)
            new_width = int(width * ratio)
            new_height = int(height * ratio)
            image = image.resize((new_width, new_height), Image.LANCZOS)
            print(f"   📐 이미지 크기 조정: {width}x{height} → {new_width}x{new_height}")
        
        # 1단계: 카테고리 분류
        print(f"   🏷️  카테고리 분류 중...")
        category_prompt = "a photo of"
        
        inputs = processor(image, text=category_prompt, return_tensors="pt").to(device, torch.float16)
        
        with torch.no_grad():
            category_ids = model.generate(
                **inputs,
                max_new_tokens=20,
                min_new_tokens=1,
            )
        
        # 프롬프트 제거하고 답변만 추출
        category_text = processor.decode(category_ids[0], skip_special_tokens=True)
        category_text = category_text.replace(category_prompt, "").strip().upper()
        
        # 카테고리 정규화
        if "HOUSE" in category_text or "HOME" in category_text or "BUILDING" in category_text:
            category = "HOUSE"
        elif "PERSON" in category_text or "PEOPLE" in category_text or "MAN" in category_text or "WOMAN" in category_text or "HUMAN" in category_text:
            category = "PERSON"
        elif "TREE" in category_text or "PLANT" in category_text:
            category = "TREE"
        else:
            category = "UNKNOWN"
        
        print(f"   📂 카테고리: {category}")
        
        # 2단계: 5가지 특성 설명 생성
        print(f"   📝 상세 설명 생성 중...")
        
        # BLIP-2는 간단한 프롬프트가 더 효과적
        description_prompt = f"Describe this {category.lower()} drawing in detail:"
        
        inputs = processor(image, text=description_prompt, return_tensors="pt").to(device, torch.float16)
        
        with torch.no_grad():
            caption_ids = model.generate(
                **inputs,
                max_new_tokens=120,
                min_new_tokens=40,
                do_sample=True,  # 샘플링 활성화
                temperature=0.7,
                top_p=0.9,
                repetition_penalty=1.5,  # 반복 방지 (중요!)
                no_repeat_ngram_size=3,  # 3단어 이상 반복 방지
            )
        
        # 프롬프트 제거하고 답변만 추출
        caption = processor.decode(caption_ids[0], skip_special_tokens=True)
        caption = caption.replace(description_prompt, "").strip()
        
        elapsed = time.time() - start_time
        print(f"   ⏱️  처리 시간: {elapsed:.1f}초")
        
        return {
            "category": category,
            "caption": caption
        }
        
    except Exception as e:
        print(f"❌ 오류 발생 ({image_path}): {e}")
        import traceback
        traceback.print_exc()
        return None

print("✅ 캡셔닝 함수 정의 완료")

✅ 캡셔닝 함수 정의 완료


## 4. 이미지 디렉토리 설정

In [6]:
# 이미지 디렉토리 설정
image_dir = r"C:\Users\helen\Desktop\kt cloud tech up\basic_project\test"
output_file = "image_captions_blip.json"

# 지원하는 이미지 확장자
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif', '.webp']

# 이미지 파일 찾기
image_files = []
for ext in image_extensions:
    image_files.extend(Path(image_dir).glob(f'*{ext}'))
    image_files.extend(Path(image_dir).glob(f'*{ext.upper()}'))

image_files = sorted(set(image_files))

print(f"📁 이미지 디렉토리: {image_dir}")
print(f"🖼️  발견된 이미지: {len(image_files)}개\n")

if len(image_files) == 0:
    print("⚠️  이미지 파일이 없습니다!")
else:
    for img in image_files:
        print(f"   - {img.name}")

📁 이미지 디렉토리: C:\Users\helen\Desktop\kt cloud tech up\basic_project\test
🖼️  발견된 이미지: 4개

   - test_나무.JPG
   - test_남자사람.JPG
   - test_여자사람.JPG
   - test_집.JPG


## 5. 캡션 생성 실행

In [7]:
# 캡션 생성
results = {}

print("\n" + "=" * 70)
print("🎨 캡션 생성 시작")
print("=" * 70 + "\n")

total_start_time = time.time()

for i, image_path in enumerate(image_files, 1):
    print(f"\n[{i}/{len(image_files)}] 처리 중: {image_path.name}")
    
    result = generate_detailed_caption(str(image_path))
    
    if result:
        results[image_path.name] = {
            "path": str(image_path),
            "category": result["category"],
            "caption": result["caption"]
        }
        print(f"✅ 완료")
        print(f"📂 카테고리: {result['category']}")
        print(f"📝 캡션: {result['caption'][:150]}{'...' if len(result['caption']) > 150 else ''}")
    else:
        print(f"⚠️  캡션 생성 실패")
    
    # GPU 메모리 정리
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

total_elapsed = time.time() - total_start_time

print("\n" + "=" * 70)
print(f"✨ 총 {len(results)}개 이미지 처리 완료!")
print(f"⏱️  전체 소요 시간: {total_elapsed:.1f}초 (평균: {total_elapsed/len(image_files):.1f}초/이미지)")
print("=" * 70)


🎨 캡션 생성 시작


[1/4] 처리 중: test_나무.JPG
   🏷️  카테고리 분류 중...
   📂 카테고리: TREE
   📝 상세 설명 생성 중...
   📂 카테고리: TREE
   📝 상세 설명 생성 중...
   ⏱️  처리 시간: 10.7초
✅ 완료
📂 카테고리: TREE
📝 캡션: trees, springtime and leaves are drawn with a simple line tool the whole sketch is shown on one page by two lines that have been moved from each other...

[2/4] 처리 중: test_남자사람.JPG
   🏷️  카테고리 분류 중...
   ⏱️  처리 시간: 10.7초
✅ 완료
📂 카테고리: TREE
📝 캡션: trees, springtime and leaves are drawn with a simple line tool the whole sketch is shown on one page by two lines that have been moved from each other...

[2/4] 처리 중: test_남자사람.JPG
   🏷️  카테고리 분류 중...
   📂 카테고리: PERSON
   📝 상세 설명 생성 중...
   📂 카테고리: PERSON
   📝 상세 설명 생성 중...
   ⏱️  처리 시간: 8.1초
✅ 완료
📂 카테고리: PERSON
📝 캡션: what is he wearing? where did the picture come from and how was it created using only a pencil or pen for example, do you know which colour of graphit...

[3/4] 처리 중: test_여자사람.JPG
   🏷️  카테고리 분류 중...
   ⏱️  처리 시간: 8.1초
✅ 완료
📂 카테고리: PERSON
📝 캡션: what is he wearin

## 6. 결과 저장

In [8]:
# JSON 파일로 저장
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"\n💾 결과 저장 완료: {output_file}")
print(f"📊 총 {len(results)}개 이미지 캡션 저장됨\n")

# 결과 미리보기
print("=" * 70)
print("📋 생성된 캡션 요약")
print("=" * 70)
for filename, data in results.items():
    print(f"\n🖼️  {filename}")
    print(f"   📂 카테고리: {data.get('category', 'UNKNOWN')}")
    print(f"   📝 캡션: {data['caption'][:100]}{'...' if len(data['caption']) > 100 else ''}")


💾 결과 저장 완료: image_captions_blip.json
📊 총 4개 이미지 캡션 저장됨

📋 생성된 캡션 요약

🖼️  test_나무.JPG
   📂 카테고리: TREE
   📝 캡션: trees, springtime and leaves are drawn with a simple line tool the whole sketch is shown on one page...

🖼️  test_남자사람.JPG
   📂 카테고리: PERSON
   📝 캡션: what is he wearing? where did the picture come from and how was it created using only a pencil or pe...

🖼️  test_여자사람.JPG
   📂 카테고리: UNKNOWN
   📝 캡션: a person standing with arms outstretched, looking towards the sky and raising their hands to stretch...

🖼️  test_집.JPG
   📂 카테고리: HOUSE
   📝 캡션: the roof, entrance and windows of a large building with an attached garage door person's sketchbook ...
